In [1]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")

In [2]:
l={}
u=list()
url= "https://www.proxynova.com/proxy-server-list/country-ISO 3166-2:IL/"
respo = requests.get(url).text
soup = BeautifulSoup(respo, "html.parser")
allproxy = soup.find_all("tr")
for proxy in allproxy:
    foo = proxy.find_all("td")
try: 
    l["ip"]=foo[0].text.replace("\n","").replace("document.write(","").replace(")","").replace("\'","").replace(";","")
except:
    l["ip"]=None
try:
    l["port"]=foo[1].text.replace("\n","").replace(" ","")
except:
    l["port"]=None
try:
    l["country"]=foo[5].text.replace("\n","").replace(" ","")
except:
    l["country"]=None
if(l["port"] is not None):
    u.append(l)
l={}
print(u)

[]


## Extract animals data from http://animalia.bio/

At this part we convert the main html pages into BeautifulSoup format.
We took three animals' classes: Mammals, Reptiles and Birds.

Each class is located in different page in the main site.

In [3]:
Mammals_url = "http://animalia.bio/mammals"
Reptiles_url = "http://animalia.bio/reptiles"
Birds_url = "http://animalia.bio/birds"

response_mammals = requests.get(Mammals_url)
response_reptiles = requests.get(Reptiles_url)
response_birds = requests.get(Birds_url)

Mammals_soup = BeautifulSoup(response_mammals.content, "html.parser")
Reptiles_soup = BeautifulSoup(response_reptiles.content, "html.parser")
Birds_soup = BeautifulSoup(response_birds.content, "html.parser")

This function gets a class page and return all animals' links which located at this specific page.

In [4]:
def list_of_url(animal_soup):
    tag_animals = animal_soup.find("div", attrs = {"class":"animals"})
    animals_links = [i['href'] for i in tag_animals.findAll("a")]
    return animals_links

This function gets current page and return the next page.

In [5]:
def get_next_page(current_page): 
    tag_next_pages = current_page.find("div", attrs = {"class":"paginator"})
    pages = [i['href'] for i in tag_next_pages.findAll("a")]
    return pages[-1]                                        

This function gets all three classes (Mammals, reptiles an Bird at BeautifulSoup format), and return a list which include all animals pages links together.
```
 Number of pages at Mammals' class: 22
 Number of pages at Reptiles' class: 6
 Number of pages at Birds' class: 12
```

In [6]:
def extract_all_animals(Mammals_soup, Reptiles_soup, Birds_soup):
    all_animals = []    # this list will contain all individuals' animals' pages
    
    """ For Mammals """
    
    all_animals = all_animals + list_of_url(Mammals_soup)    # Extract the animals from the fisrt Mammals page  
    next_Mammals_soup = Mammals_soup    # Save the first page in new variable
    for i in range(1, 22):    # According to the number of pages for each class
        new_url = get_next_page(next_Mammals_soup)
        response_new_url = requests.get(new_url)
        next_Mammals_soup = BeautifulSoup(response_new_url.content, "html.parser")    # Convert to BeautifulSoup format
        all_animals = all_animals + list_of_url(next_Mammals_soup)    # Extract the animals from the current Mammals page
        
    """ For Reptiles """
        
    all_animals = all_animals + list_of_url(Reptiles_soup)
    next_Reptiles_soup = Reptiles_soup
    for i in range(1, 6):
        new_url = get_next_page(next_Reptiles_soup)
        response_new_url = requests.get(new_url)
        next_Reptiles_soup = BeautifulSoup(response_new_url.content, "html.parser")
        all_animals = all_animals + list_of_url(next_Reptiles_soup)
    
    """ For Birds """
    
    all_animals = all_animals + list_of_url(Birds_soup)
    next_Birds_soup = Birds_soup
    for i in range(1, 12):
        new_url = get_next_page(next_Birds_soup)
        response_new_url = requests.get(new_url)
        next_Birds_soup = BeautifulSoup(response_new_url.content, "html.parser")
        all_animals = all_animals + list_of_url(next_Birds_soup)
        
    return all_animals    

In [7]:
links_all_animals = extract_all_animals(Mammals_soup, Reptiles_soup, Birds_soup)


In [8]:
len(links_all_animals)

1597

In [9]:
with open('links_animals.txt', 'w') as f:
    for link in links_all_animals:
        f.write(link + '\n')